# Conditional Random Field - a Review

Conditional Random Field has been widely used as an efficient post-processing method for image segmentation. 
There are several python packages for dealing with Conditional Random Field:
<ul>
	<li><a href='https://github.com/lucasb-eyer/pydensecrf/'>pydensecrf</a> is a Python wrapper to <a href='http://web.archive.org/web/20161023180357/http://www.philkr.net/home/densecrf'>
 Philipp Krähenbühl's
 dense (fully connected) CRFs with gaussian edge potentials.</a>
	<li> PyStruct library also has <a href='https://pystruct.github.io/generated/pystruct.models.GraphCRF.html#pystruct.models.GraphCRF'>
GraphCRF module</a> allows us to use on arbitrary graph for classification.
 However, for image segmentation post-processing, I prefer to use pydensecrf.
	<li> <a href='https://python-crfsuite.readthedocs.io/en/latest/'>python-crfsuite</a> is a python binding to 
<a href='https://github.com/chokkan/crfsuite'>crfsuite</a>. 
		


In this post, we consider Conditional Random Field for image post-processing.

Consider a random field $X$, which ranges over possible pixel labelings $\mathcal{L} = \{l_1,l_2,...,l_L\}$, and is defined over a set of variables $\{X_1,...,X_n\}$ where $n$ is the size of the input images. Then $X_j$ is the label assigned to the $jth$ pixel. Let D be the observed data sequence, and $P_i, I_i$ denote the position and color vector of the $ith$ pixel, respectively.

A common Conditional Random Field model is where the energy is of the form:

$$E(x|D)=\sum_i\psi_u(x_i)+\sum_{i<j}\psi_p(x_i,x_j)$$
where $i,j$ range from $1$ to $n$.

The unary potentials are of the form
$$\psi_u(x_i) = -\log P(X_i=x_i)$$

For post-processing, the unary potential $\psi_u(x_i)$ can the softmax output of a neural network. 

Our pairwise potential for pixels is of the form

$$\psi_p(x_i,x_j) = w^{(1)}\exp\left( -\frac{|P_i-P_j|^2}{2\theta_\alpha^2}-\frac{|I_i-I_j|^2}{2\theta^2_\beta} \right)+w^{(2)}\exp\left(-\frac{|P_i-P_j|^2}{2\theta_\gamma^2}\right) $$

where $\mu(x_i,x_j)$ is the label compabitility function. A common choice is to use the Potts model $\mu(x_i,x_j)=[x_i\neq x_j]$ that captures the compatibility between different pairs of labels, and $w$ is the weight. 

The first exponential term is the appearance kernel. Its is inspired by the heuristic that pixels in the same class tend to have similar in color and in nearby position. The degree of nearness and color similarity are controlled by parameters $\theta_\alpha, \theta_\beta$, respectively. 

The second exponential term is the smoothness kernel that helps removing small isolated regions. (see Textonboost by Shotton)

For the model above, only 1st and 2nd order are considered. However, higher potentials is possible (see *Higher-order models in Computer Vision* by Pushmeet Kohli and Carsten Rother, or *Robust Higher Order Potentials for Enforcing Label Consistency*)

## Superpixel enhanced Conditional Random Field 

Superpixel enhanced Conditional Random Field  can be considered as a special case of higher order Conditional Random Field. 
Tutorial about superpixel on <a href='https://www.pyimagesearch.com/2014/07/28/a-slic-superpixel-tutorial-using-python/'>
pyimagesearch</a>. Note that the parameter sigma in <font color="#00bfff"> segments = slic(image, n_segments = numSegments, sigma = 5)</font> 
control the smoothness of the superpixel. When using super pixel enhanced CRF, we should tune sigma according to the image nature.
In some medical image, the boundary maybe very rough. In such a case, lower sigma should be used.

The corresponding Gibbs energy of the model is

$$E(x|D)=\sum_i\psi_u(x_i)+\sum_{i<j}\psi_p(x_i,x_j) + \sum_{i<j}\psi_{sp}(x_i,x_j)$$
where $i,j$ range from $1$ to $n$.

where the unary potential $\psi_u(x_i)$, pairwise potential $\psi_p(x_i,x_j)$ are similarly defined as before, and $\psi_{sp}(x_i,x_j)$ is a super pixel pairwise potential. 

In this case, we pre-process the original images with some unsupervised segmentation and denote by $s_i$ the segment index of pixel i. From this, we can create a segmented image where each pixel i takes the average RGB value $C_{s_i}$ of the superpixel that it belongs.  The super pixel pairwise potential can be defined as in *super pixel-enhanced pairwise Conditional Random Field for semantic segmentation* by Li Sulimowicz, Ishfaq Ahmadc, Alexander Aved:

$$\psi_{sp}(x_i,x_j)=\mu(x_i, x_j)(\theta_p+\theta_v\exp(-\theta_\beta |C_{s_i}-C_{s_j}|^2))$$
where $\mu(x_i, x_j)=[x_i\neq x_j]$ as before.

Note that if pixel $i,j$ belongs to the same super-pixel then 
$$\psi_{sp}(x_i,x_j)=\mu(x_i, x_j)(\theta_p+\theta_v)$$

Another potential function form introduced in the paper is

$$\psi_{sp}(x_i,x_j) = w^{(1)}_{sp}\exp\left( -\frac{|P_i-P_j|^2}{2{\theta^s}_\alpha^2}-\frac{|C_{s_i}-C_{s_j}|^2}{2{\theta^s}^2_\beta} \right)$$


Related papers:

- TextonBoost: Joint Appearance, Shape and
Context Modeling for Multi-Class Object
Recognition and Segmentation